In [178]:
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt,pow
from scipy.stats import multivariate_normal
import pickle

np.random.seed(42)

In [179]:
class FileReader:
    @staticmethod
    def read_parameters():
        file = open("parameter.txt");
        arr = []
        for line in file.readlines():
            arr.append(float(line))
        
        return arr[0],arr[1],arr[2]    
    
    @staticmethod
    def read_file(filename):
        try:
            file = open(filename)
            
            bits = []
            
            while(True):
                bit = file.read(1)
                if not bit:
                    break
                else:
                    bits.append(int(bit))
            file.close();
            
            return np.array(bits)
        except:
            print("I/O Error")

In [180]:
train_bits = FileReader.read_file("train.txt")
test_bits = FileReader.read_file("test.txt")
hk,hk_1,var = FileReader.read_parameters() 

## Initializing Variables

In [181]:
n = 2
l = 2
number_of_clusters = int(pow(2,n+l-1))

In [182]:
def channel_out(ik,ik_1):
    return hk*ik + hk_1*ik_1 + np.random.normal(0,var)

In [183]:
def gen_co_var_matrix(data):
    dataset = np.array(data)
    mean_arr = np.array([np.mean(dataset[:,i]) for i in range(dataset.shape[1])])
    co_var_matrix = np.zeros((dataset.shape[1],dataset.shape[1]))
    dataset_len = dataset.shape[0]

    for i in range(dataset.shape[1]):
        for j in range(dataset.shape[1]):
            X = dataset[:,i] - mean_arr[i]
            Y = dataset[:,j] - mean_arr[j]
            Z = np.multiply(X,Y)
            total = np.sum(Z,axis=0)
            co_var_matrix[i][j] = total/(dataset_len-1)

    return co_var_matrix

In [184]:
def fit(train_bits):
    mean = [[0,0]]*number_of_clusters
    co_var = [np.zeros((n,n))]*number_of_clusters

    cluster_count = np.zeros(number_of_clusters)

    prior_prob = np.zeros(number_of_clusters)
    transition_prob = np.zeros((number_of_clusters,number_of_clusters))

    cluster_count = [0]*number_of_clusters
    cluster_wise_data = []

    for _ in range(number_of_clusters):
        cluster_wise_data.append([])
        
    ik_1 = 0
    ik_2 = 0
    xk_1 = channel_out(ik_1,ik_2)
    size = len(train_bits)
    prev = 0
    
    total = 0
    
    for ik in train_bits:
        xk =  channel_out(ik,ik_1)
        cluster = [xk,xk_1]
        cluster_id = int(ik*(pow(2,2)) + ik_1*(pow(2,1)) + ik_2*(pow(2,0)))
        
        cluster_count[cluster_id] += 1
        transition_prob[prev][cluster_id] += 1
        
        ik_2 = ik_1
        ik_1 = ik
        xk_1 = xk
        prev = cluster_id
        
        mean[cluster_id] = np.add(mean[cluster_id],cluster)
        cluster_wise_data[cluster_id].append(cluster)
        cluster_count[cluster_id] += 1
        
        total += 1
                
        if(total%100000==0):
            print("Done: " + str(total/size) + "%")
    
    
    for i in range(number_of_clusters):
        prior_prob[i] = cluster_count[i]/total
        mean[i] = mean[i]/cluster_count[i]
        co_var[i] = gen_co_var_matrix(cluster_wise_data[i])
    
    transition_prob = transition_prob/total
    
    return mean,co_var,prior_prob,transition_prob

In [185]:
mean,co_var,prior_prob,transition_prob = fit(train_bits)

Done: 0.0100000010000001%
Done: 0.0200000020000002%
Done: 0.0300000030000003%
Done: 0.0400000040000004%
Done: 0.0500000050000005%
Done: 0.0600000060000006%
Done: 0.0700000070000007%
Done: 0.0800000080000008%
Done: 0.0900000090000009%
Done: 0.100000010000001%
Done: 0.1100000110000011%
Done: 0.1200000120000012%
Done: 0.1300000130000013%
Done: 0.1400000140000014%
Done: 0.1500000150000015%
Done: 0.1600000160000016%
Done: 0.1700000170000017%
Done: 0.1800000180000018%
Done: 0.1900000190000019%
Done: 0.200000020000002%
Done: 0.2100000210000021%
Done: 0.2200000220000022%
Done: 0.2300000230000023%
Done: 0.2400000240000024%
Done: 0.2500000250000025%
Done: 0.2600000260000026%
Done: 0.2700000270000027%
Done: 0.2800000280000028%
Done: 0.2900000290000029%
Done: 0.300000030000003%
Done: 0.3100000310000031%
Done: 0.3200000320000032%
Done: 0.3300000330000033%
Done: 0.3400000340000034%
Done: 0.3500000350000035%
Done: 0.3600000360000036%
Done: 0.3700000370000037%
Done: 0.3800000380000038%
Done: 0.3900000

In [186]:
def generate_output_data(test_bits):
    output_data = []
    ik_2 = 0
    ik_1 = 0
        
    xk_2 = channel_out(ik_1,ik_2)
    xk_1 = channel_out(ik_1,ik_2)
        
    cluster = [xk_1,xk_2]
    output_data.append(cluster)
        
    for ik in test_bits:
        xk = channel_out(ik,ik_1)
        cluster = [xk,xk_1]
        output_data.append(cluster)
        
        xk_1 = xk
        ik_1 = ik
        
        
        
    return output_data

In [217]:
output_data = generate_output_data(test_bits)

In [218]:
def predict_by_cluster(output_data):
    num = len(output_data)
    cost = np.zeros((num,number_of_clusters))
    
    for i in range(num):
        sample = output_data[i]
        
        for j in range(number_of_clusters):
            if i == 0:
                cost[i][j] = 0
            else:
                cluster_center = mean[j]
                cost[i][j] = sqrt(pow(sample[0] - cluster_center[0], 2) + pow(sample[1] - cluster_center[1], 2))
                cost[i][j] += min(cost[i-1])
    
    recovered = []
    for i in range(num-1,0,-1):
        cluster = np.argmin(cost[i])
        
        if(cluster<4):
            recovered.append(0)
        else:
            recovered.append(1)

    recovered.reverse()
        
    return recovered

In [219]:
pred = predict_by_cluster(output_data)

In [220]:
def calculate_accuracy(d1,d2):
    size = len(d1)
    
    acc = 0
    
    for i in range(size):
        if d1[i] == d2[i]:
            acc += 1
        else:
            print(i,d1[i],d2[i])
    
    return acc/size

In [221]:
# Method 2 Accuracy
calculate_accuracy(test_bits,pred)

3 0 1
6 0 1
7 0 1
8 0 1
18 1 0
22 0 1
25 0 1
45 0 1
47 1 0
48 1 0
51 0 1
52 0 1
67 0 1
68 0 1
74 0 1
76 0 1
82 0 1
97 0 1


0.82

In [222]:
def predict_by_prob(output_data):
    num = len(output_data)
    cost = np.zeros((num,number_of_clusters))
    for i in range(num):
        sample = output_data[i]
        for j in range(number_of_clusters):
            if i == 0:
                cost[i][j] = 0
            if i == 1:
                prev = np.argmax(cost[i-1])
                cost[i][j] = np.log(0.1+transition_prob[prev][0]*multivariate_normal.pdf(sample,mean[j],co_var[j]))
            else:
                prev = np.argmax(cost[i-1])
                cost[i][j] = np.log(0.1+transition_prob[prev][j]*multivariate_normal.pdf(sample,mean[j],co_var[j]))
                cost[i][j] += max(cost[i-1])
    
    recovered = []
    for i in range(num-1,0,-1):
        cluster = np.argmax(cost[i])
        
        if(cluster<4):
            recovered.append(0)
        else:
            recovered.append(1)

    recovered.reverse()
        
    return recovered

In [223]:
pred = predict_by_prob(output_data)

In [224]:
# Method 2 Accuracy
calculate_accuracy(test_bits,pred)

3 0 1
6 0 1
7 0 1
8 0 1
22 0 1
25 0 1
45 0 1
47 1 0
48 1 0
51 0 1
52 0 1
67 0 1
68 0 1
74 0 1
76 0 1
82 0 1
97 0 1


0.83